# Computing Embeddings of Recipes

Using the Ollama embedding tutorial here: https://ollama.com/blog/embedding-models
Note: Requires ChromaDB version 0.5.3 due to later versions crashing. 

In [1]:
import json
import os 
import ollama
import chromadb
import gc

In [2]:
# load the recipe JSONs as `documents`
json_recipe_files = [f for f in os.listdir("json_chapters")]
json_recipe_files = [os.path.join("json_chapters",file) for file in json_recipe_files]
documents = []
ids =[]

for file in json_recipe_files:
    with open(file, "r") as f:
        data = json.load(f)

        for recipe in data:
            text = f"Title: {recipe['title']} \n Instructions: {recipe['instructions']}"
            documents.append(text)
            ids.append(recipe['recipe_id'])


In [3]:
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="recipes")

# store each document in a vector embedding database
i = 0
for recipe_id, recipe in zip(ids, documents):
  if i % 100 == 0:
    print(f"Embedding recipe: {i+1}.")
  response = ollama.embed(model="mxbai-embed-large:latest", input=recipe)
  embeddings = response["embeddings"]
  # break
  collection.add(
    ids=[str(recipe_id)],  # use the recipe ID from the JSON instead
    embeddings=embeddings[0],
    documents=[recipe]
    # can add recipe metadata here such as ingredients and the chapter title
  )
  i += 1

Embedding recipe: 1.


Add of existing embedding ID: 127
Insert of existing embedding ID: 127
Add of existing embedding ID: 138
Insert of existing embedding ID: 138
Add of existing embedding ID: 129
Insert of existing embedding ID: 129
Add of existing embedding ID: 130
Insert of existing embedding ID: 130
Add of existing embedding ID: 131
Insert of existing embedding ID: 131
Add of existing embedding ID: 132
Insert of existing embedding ID: 132
Add of existing embedding ID: 133
Insert of existing embedding ID: 133
Add of existing embedding ID: 134
Insert of existing embedding ID: 134
Add of existing embedding ID: 136
Insert of existing embedding ID: 136
Add of existing embedding ID: 137
Insert of existing embedding ID: 137
Add of existing embedding ID: 138
Insert of existing embedding ID: 138


Embedding recipe: 101.


Add of existing embedding ID: 2373
Insert of existing embedding ID: 2373
Add of existing embedding ID: 2396
Insert of existing embedding ID: 2396


Embedding recipe: 201.


...

Add of existing embedding ID: 2123
Insert of existing embedding ID: 2123


Embedding recipe: 2501.


Add of existing embedding ID: 2313
Insert of existing embedding ID: 2313
Add of existing embedding ID: 2317
Insert of existing embedding ID: 2317


Embedding recipe: 2601.


Add of existing embedding ID: 2367
Insert of existing embedding ID: 2367


In [5]:
# an example input
input = "How to make stock?"

# generate an embedding for the input and retrieve the most relevant doc
response = ollama.embed(
  model="mxbai-embed-large",
  input=input
)
results = collection.query(
  query_embeddings=[response["embeddings"][0]],
  n_results=2
)
data = results['documents'][0][0]

In [7]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama3",
  prompt=f"Using this data: {data}. Respond to this prompt: {input}"
)

print(output['response'])

Based on the provided data, here is a step-by-step guide on how to make ordinary aspic stock:

**Materials needed:**

* A very clean and well-tinned stockpot or stewpan
* 4 lbs of strung knuckle of veal
* 3 calf's feet, boned and blanched
* 3 lbs of strung knuckle of beef
* 1 lb of fresh pork rind, well cleaned and with fat removed
* 3 lbs of veal bones, well broken and blanched
* Cold water (approximately 8 quarts)
* Salt (1 oz)

**Mode of Procedure:**

1. Put the meats (veal knuckle, calf's feet, beef knuckle, pork rind, and veal bones) in the prepared stockpot or stewpan.
2. Add 8 quarts of cold water to the pot.
3. Boil the mixture, then skim off any impurities that rise to the surface.
4. Once the stock has been well-skimmed, add 1 oz of salt and stir it into the liquid.
5. Allow the stock to simmer or cook on low heat for a period of time, ensuring that all the flavors are fully infused into the aspic.

**Tips:**

* Use a very clean pot to prevent any contamination or spoilage of